##### CSCI 303
# Introduction to Data Science
<p/>
### Web Scraping

Credits: Many thanks to Zachary Smialek for this Lecture/Notebook Idea!


## This Lecture
---
- Overview of HTML
- Useful Libraries
- What is Scraping and Web Crawling
- Issues

# Background
---

HTML (HyperText Markup Language) is the fundmental building block of the Web. The purpose of HTML is describe to a webpage's appearance. HTML is context free grammer makes parsing a bit different. Quick note regular expressions will not do.

# What HTML looks like

# HTML Tags
___

The purpose of an HTML tags is to provides content for a given webpage. The structure of the tags are start < TAG> and end < /TAG>. The <, >, and / characters are called delimiters, function is to tell the parser that enclosed infomation is an HTML element. When we are scraping infomation we use HTML tags to extract the information.

Lets look at a snippet of html.

< p>This is some text in a paragraph.< /p>  
*note there is a space in the tags the notebook automatically render html.  

We can use the < p> < /p> extract text from a paragraph on a webpage. When parsed the infomation will be "This is some text in a paragraph".

# HTML Tags Examples
___
There are many different tags categories:
- Structure Tags
- Head Elements
- Body Elements
- Paragraph / Line Breaks Tags
- Link and Image

## Scraping
---
Scraping is the act of extracting useful data from a website normally done autonomously. There are different ways to achieve this using Python! Overall favorite is making a request and then parse the HTML response.

# Useful Libraries
---
urllib2 /  urllib.request - Used to interact with URLS: opening, authentication, redirects, etc. This library varies between verisons of python 2.7 and 3.6

BeautifulSoup - Used to extract information from HTML and XML files.

Scrapy - A library that intergrates scraping and crawling, more robust then BeautifulSoup.

# Lets take a look at wikipedia page
___
We can have look at all the data we can collect on a given wikipedia page.

In [ ]:
# Setting things up!
import numpy as np
import pandas as pd

from pandas import Series, DataFrame
import urllib.request
from bs4 import BeautifulSoup

import time
import os

webpage = "https://en.wikipedia.org/wiki/List_of_state_and_union_territory_capitals_in_India"

#This returns HTTPResponse object need to read and  
#decode to the see the contents.
html_bytes = urllib.request.urlopen(webpage)

#This is the soup object that will parse th HTML page
soup = BeautifulSoup(html_bytes,"html5lib")

In [ ]:
# Back to the tags, one can get at the data in the tags
# soup.<tag>

# Lets get the title of the webpage
soup.title

#Removes the tags
soup.title.string

#Here is a list of different tags: https://www.w3schools.com/tags/

# More cool stuff with beautiful soup
---
Lets look into HTML attributes and why they are useful for scraping. We will look at < a href> tag, this tag contains a link on the page. This is an example on how to get all the HTML attribute that is href. Quick side note: href is short for Hypertext REFerence.

In [ ]:
# To get an HTML Attributes
#soup.a

# But how do we get the href, have no fear
all_links = soup.find_all("a")

# Prints all the links that have a href atrributes and filter out bad links
href_links = [link.get('href') for link in all_links if link.get('href') is not None]
#href_links

# What if we wanted to scrape data out of a table?
---
Once again we need to understand what HTML tags we need to work with. HTML uses the < table> to define a table but we need the rows, columns, and header. You guessed it there are tags for that! The tags in question are < th> table header, < tr> is the table row, and < td> is the table cell. 

In [ ]:
# This is an example how to extract data from a table, this is a specific table on the webpage.
data = []
cleaned = []

# We want to find the table in the HTML: we can do this by using the inspect tool through the
# browser. Takes a little time but it does the trick. There also chrome plugins that will aid in
# this proces.
table = soup.find("table", attrs={"class" : "wikitable"})

# One can also use CSS Selectors.
#table=soup.select('th , td , #firstHeading')

table_body = table.find('tbody')

table_headers = []
header_row = []
rows = table_body.find_all('tr')

for header in table.find_all('th'):
    if header.get('scope') == 'col':
        table_headers.append(header.text.replace('\n',' '))
    
    if header.get('scope') == 'row':
        header_row.append(header.text)

        
# We need to iterate over the then the cols which is the cell data. This throw it into a list     
for row in rows:
    cols = row.find_all('td')
    cols = [column_data.text.strip().replace('\n',' ') for column_data in cols]
    
    if cols != []:
        data.append([ele for ele in cols if cols])

# This to make the table look like what it looks like on the wiki page.
counter=0
for row_info in data:
    if row_info != []:
        row_info.insert(0,header_row[counter])
        row_info.insert(0,counter)
        counter += 1

wiki_table = pd.DataFrame.from_records(data,columns=table_headers)
wiki_table

# Wonder if we can download images? Yes WE CAN!
___
Python is wonderful right! Let's take a look a simple way to grab images from a webage.

In [ ]:
def file_path(file_name):
    '''
        Return a the file path where the image will be saved to.
        ~/csci303/lectures/scraped_imgs/file_name if using the init file structure 
        
        or the ~/path_of_notebook/scraped_imgs/filename
    '''
    return os.getcwd()+"/scraped_imgs/"+file_name

In [ ]:
'''
    This a quick example how to download images for a webpage.
    Follows the same pattern we have been doing, grab all the 
    the tags in question for this example we are looking at the
    img tag.
'''

domain = "https://en.wikipedia.org"

delimiter = '/'
outside_domain_token = '//'

#First we need to get the src URL for the images and throw out dups.
img_links = list(set([link['src'] for link in soup.find_all('img')]))

for link in img_links:
    
    #parsing the file_name from the URL
    file_name = link.split(delimiter)[-1]
    
    # We have to check if the file lives in domain or not.
    #
    # not in domain //some_url.com/sdfds/file
    # /dsfsd/fds/filename if file lives in the domain 
    
    if outside_domain_token in link:
        url = "https:" + link
    else:
        url = domain + link
        
    urllib.request.urlretrieve(url,file_path(file_name))
    print("Downloaded: " + file_name)
    print("File path: " + file_path(file_name) + "\n")
    
    #To slow down the request rate
    time.sleep(1.5)